In [2]:
import os

from openai import OpenAI
from transformers.utils.versions import require_version


/home/sc/anaconda3/envs/aitongue/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
instruction_prompt = """
请根据给定的舌象图片及其标签数据，生成一段描述舌象特征的文字。描述应包含以下必填项：
- 舌质颜色
- 舌体大小（胖、胖瘦适中、瘦）
- 舌面湿润或干燥 (湿润、偏润、润燥适中、偏燥、干燥)
- 舌边有无齿痕 （有齿痕、无齿痕）
- 舌面有无裂纹、有无点刺 （有裂纹、无裂纹、有点刺、无点刺，两种症状都需要描述）
- 舌体老嫩 （嫩、偏嫩、老嫩适中、偏老、老）
- 舌苔覆盖范围 （全苔、舌根部、舌中部、舌尖部、舌左部、舌右部、无偏苔）
- 舌苔表现形式

舌苔的表现形式必须按照以下规则描述：
- 舌苔颜色 + 厚薄 + 腻腐（如：苔黄厚腻）

"""


## LLaMAFactory API调用示例(Recommend)

In [3]:
client = OpenAI(
        api_key="{}".format(os.environ.get("API_KEY", "0")),
        base_url="http://localhost:{}/v1".format(os.environ.get("API_PORT", 8000))
    )
messages = []
messages.append(
    {
        "role": "system",
        "content": instruction_prompt
    }
)
messages.append(
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "描述这个舌象图片"},
            {
                "type": "image_url",
                "image_url": {"url": "/home/sc/LLM_Learning/AI_TongueDiag/data/images/0003-1.jpg"}
            }
        ]
    }
)

## 非流式输出

In [ ]:
result = client.chat.completions.create(messages=messages, model="Qwen2-VL-7B-Instruct")

messages.append(result.choices[0].message)
print("Round 1:", result.choices[0].message.content)

## 流式输出

In [ ]:
import time
# create variables to collect the stream of chunks
start_time = time.time()

result = client.chat.completions.create(messages=messages, model="Qwen2-VL-7B-Instruct", stream=True)

collected_chunks = []
collected_messages = []
# iterate through the stream of events
for chunk in result:
    chunk_time = time.time() - start_time  # calculate the time delay of the chunk
    collected_chunks.append(chunk)  # save the event response
    chunk_message = chunk.choices[0].delta  # extract the message
    collected_messages.append(chunk_message)  # save the message
    print(f"Message received {chunk_time:.2f} seconds after request: {chunk_message}")  # print the delay and text

# print the time delay and text received
print(f"Full response received {chunk_time:.2f} seconds after request")


In [10]:
full_reply_content = ''
for m in collected_messages:
    if m.content is not None:
        full_reply_content += m.content
    else:
        full_reply_content +=  ''

print(f"Full conversation received: {full_reply_content}")

Full conversation received: 根据您输入的图片，模型分析的舌象结果是：
    -舌质颜色：淡红；
    -舌体大小：瘦；
    -舌面湿润程度：润燥适中；
    -舌边：无齿痕；
    -舌面：无裂纹、无点刺；
    -舌质老嫩程度：老嫩适中；
    -舌苔主要覆盖情况：无偏苔；
    -舌苔表现形式：苔偏白厚薄适中无腻腐
    
    


## vllm版本 API调用

In [ ]:
import base64
import requests
import json

def encode_image(image_path):       # 编码本地图片的函数
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

# 1. url
url = "http://localhost:{}/v1/chat/completions".format(os.environ.get("API_PORT", 8000))

# 2. data
image_path = "/home/sc/LLM_Learning/AI_TongueDiag/data/images/0003-1.jpg"
base64_image = encode_image(image_path)
data = {
    "model": "Qwen2-VL-7B-Instruct",
    "messages": [
            {
        "role": "system",
        "content": instruction_prompt,
    },
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "描述这个舌象图片"},
            {
                "type": "image_url",
                "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"},
            }
        ]
    }
    ]
}


# 3.将字典转换为 JSON 字符串
json_payload = json.dumps(data)

# 4.发送 POST 请求
headers = {'Content-Type': 'application/json'}
response = requests.post(url, data=json_payload, headers=headers)

# 5.打印响应内容
print(response.json())

In [13]:
print(response.json()['choices'][0]['message']['content'])

根据您输入的图片，模型分析的舌象结果是：
    -舌质颜色：淡红；
    -舌体大小：瘦；
    -舌面湿润程度：润燥适中；
    -舌边：无齿痕；
    -舌面：有裂纹、有点刺；
    -舌质老嫩程度：老嫩适中；
    -舌苔主要覆盖情况：无偏苔；
    -舌苔表现形式：苔偏白厚薄适中无腻腐
    
    


## Huggingface-FastAPI API调用

In [6]:
client = OpenAI(
        api_key="{}".format(os.environ.get("API_KEY", "0")),
        base_url="http://localhost:{}/v1".format(os.environ.get("API_PORT", 8000))
    )
messages = []
messages.append(
    {
        "role": "system",
        "content": instruction_prompt
    }
)
messages.append(
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "描述这个舌象图片"},
            {
                "type": "image",
                "image": "/home/sc/LLM_Learning/AI_TongueDiag/data/images/0003-1.jpg"
            }
        ]
    }
)

In [ ]:
result = client.chat.completions.create(messages=messages, model="Qwen2-VL-7B-Instruct")

messages.append(result.choices[0].message)
print("Round 1:", result.choices[0].text)